In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
import findspark
findspark.init()
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame as SparkDataFrame
from pyspark.sql.types import IntegerType, FloatType, StringType, StructType
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, jaccard_score, confusion_matrix
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [13]:
def preprocessing(columns_used_for_grouping: list, Year: int, Month: int):
  spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
  columns = ['lon', 'lat', 'Date'] + columns_used_for_grouping
  schema = StructType()
  for i in columns:
    if i == "Date":
      schema = schema.add(i, IntegerType(), True)
    else:
      schema = schema.add(i, FloatType(), True)
  nasa = spark.read.format('csv').option("header", True).schema(schema).load('/content/drive/MyDrive/BigMess/NASA/NASA.csv')
  nasa.createOrReplaceTempView("nasa")
  nasa_ym = spark.sql("""
        SELECT
        CAST(SUBSTRING(CAST(Date AS STRING), 1, 4) AS INT) AS Year,
        CAST(SUBSTRING(CAST(Date AS STRING), 5, 2) AS INT) AS Month,
        n.*
        FROM nasa n
        """)
  nasa_ym = nasa_ym.drop("Date")
  nasa_ym.createOrReplaceTempView("nasa_ym")
  query = f"""
  SELECT
  *
  FROM nasa_ym
  WHERE Year = {Year} AND Month = {Month}
  ORDER BY lon, lat, Year, Month
  """
  SparkDataFrame_Year_Month = spark.sql(query)
  df_Year_month = SparkDataFrame_Year_Month.toPandas()
  NASA_an = pd.read_csv('/content/drive/MyDrive/BigMess/NASA/NASA_an.csv', sep=';')
  NASA_an['pustynia_i_step'] = NASA_an.pustynia + NASA_an.step
  NASA_Year_Month = df_Year_month[["lon", "lat"] + columns_used_for_grouping].merge(NASA_an, left_on=['lon','lat'], right_on = ['lon','lat'], how='inner')
  X_test = NASA_Year_Month[columns_used_for_grouping]
  y_test = NASA_Year_Month[["pustynia_i_step"]]
  return X_test, y_test

In [14]:
class ClassificationModelEvaluator:
    def __init__(self, X_test, y_test):
        self.X_test = X_test
        self.y_test = y_test
    def evaluate_model(self, model):
        y_pred = model.fit_predict(self.X_test)
        accuracy = accuracy_score(self.y_test, y_pred)
        precision = precision_score(self.y_test, y_pred)
        recall = recall_score(self.y_test, y_pred)
        f1 = f1_score(self.y_test, y_pred)
        jaccard = jaccard_score(self.y_test, y_pred)
        confusion_mat = confusion_matrix(self.y_test, y_pred)
        metrics_dict = {
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'F1 Score': f1,
            'Jaccard Score': jaccard,
            'Confusion Matrix': confusion_mat
        }
        return metrics_dict

In [18]:
# Działanie:
# X_test, y_test = preprocessing(['Rainf', 'Evap', 'AvgSurfT','Albedo', 'SoilT_10_40cm', 'GVEG', 'PotEvap', 'RootMoist', 'SoilM_100_200cm'], 2023, 7)
# evaluator = ClassificationModelEvaluator(X_test, y_test)
# metrics = evaluator.evaluate_model(KMeans(n_clusters = 2, init='k-means++', random_state = 123))
# print(metrics)

{'Accuracy': 0.722, 'Precision': 0.5304659498207885, 'Recall': 0.9487179487179487, 'F1 Score': 0.6804597701149426, 'Jaccard Score': 0.5156794425087108, 'Confusion Matrix': array([[213, 131],
       [  8, 148]])}


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
